# Atividade 03 - Aprendizado de Máquina (IA048)

Álvaro Tona Ribas Cruz - 239520

Antonio César de Andrade Júnior - 245628

## 0 - Imports

In [ ]:
pip install medmnist

In [ ]:
pip install skorch

In [ ]:
from medmnist import BloodMNIST

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetClassifier
from skorch.callbacks import EarlyStopping
from torchvision.transforms import ToTensor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelBinarizer

## (a) MLP

CROSSVAL

In [ ]:
train = BloodMNIST(split="train", download=True, size=28)
test = BloodMNIST(split="test", download=True, size=28)

x_train = train.imgs.reshape(-1, 3*28*28)
x_test = test.imgs.reshape(-1, 3*28*28)

lb = LabelBinarizer()
y_train = lb.fit_transform(train.labels.ravel())
y_test = lb.transform(test.labels.ravel())


def create_model(hidden_layer_size=64, activation='relu'):
    model = tf.keras.models.Sequential([
        Dense(hidden_layer_size, activation=activation, input_dim=x_train.shape[1]),
        Dense(8, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=["accuracy"])
    return model

model = KerasClassifier(build_fn=create_model, epochs=300, batch_size=32, verbose=0)



hidden_layer_size= [64 ,128, 256]
activation= ['relu', 'tanh', 'sigmoid']

param = dict(model__hidden_layer_size = hidden_layer_size, model__activation=activation)

grid = GridSearchCV(estimator=model, param_grid=param, cv=3)
result = grid.fit(x_train, y_train)

print("Melhores Parâmetros: %s - Score: %.4f" % (result.best_params_, result.best_score_))

print("Todos os resultados:")
for mean, std, params in zip(grid.cv_results_['mean_test_score'], grid.cv_results_['std_test_score'], grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) para %r" % (mean, std * 2, params))

Teste

In [ ]:
train = BloodMNIST(split="train", download= True, size = 28)

val = BloodMNIST(split="val", download= True, size = 28)

test = BloodMNIST(split="test", download= True, size = 28)

x_train = train.imgs.reshape(-1, 3*28*28)
lb = LabelBinarizer()
y_train = lb.fit_transform(train.labels.ravel())


x_val = val.imgs.reshape(-1, 3*28*28)
lb = LabelBinarizer()
y_val = lb.fit_transform(val.labels.ravel())

mlp = tf.keras.models.Sequential()

callback = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0.001,
    patience=25,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

mlp.add(Dense(256, activation='relu'))
mlp.add(Dense(8, activation='softmax'))

mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=["accuracy"])

mlp.fit(x_train, y_train, epochs=300, batch_size=32, callbacks=[callback],verbose =0, validation_data=(x_val, y_val))

x_test = test.imgs.reshape(-1, 3*28*28)
lb = LabelBinarizer()
y_test = lb.fit_transform(test.labels.ravel())

y_pred = mlp.predict(x_test)
y_pred_indices = np.argmax(y_pred, axis=1)
y_pred_one_hot = np.eye(np.max(y_pred_indices) + 1)[y_pred_indices]

y_pred_indices

acc = accuracy_score(y_test, y_pred_one_hot)

print(acc)

## (b) CNN simples

Rede Neural Convolucional.

Ela contém as camadas pedidas no exercício, e as variáveis que devem ser controladas para avaliação do desempenho.

In [ ]:
train = BloodMNIST(split="train", download= True, size = 28)

val = BloodMNIST(split="val", download= True, size = 28)

test = BloodMNIST(split="test", download= True, size = 28)

x_train = train.imgs / 255
x_val = val.imgs / 255
x_test = test.imgs / 255

lb = LabelBinarizer()
y_train = lb.fit_transform(train.labels.ravel())
y_val = lb.fit_transform(val.labels.ravel())
y_test = lb.transform(test.labels.ravel())


def build_model(hp):
    kernel_size = hp.Choice('kernel_size', ['2,2', '4,4', '8,8', '16,16'])
    kernel_size = tuple(map(int, kernel_size.split(',')))

    num_filters = hp.Choice('num_filters', [64,128,256])

    input_tensor = Input(shape=(28, 28, 3))
    network = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=1, padding='same')(input_tensor)
    network = MaxPooling2D((1,1))(network)
    network = BatchNormalization()(network)
    network = ReLU()(network)
    network = GlobalAveragePooling2D()(network)
    output_tensor = Dense(8, activation='softmax')(network)
    model = Model(inputs=input_tensor, outputs=output_tensor)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='output',
    project_name='BloodMNISTGridSearch'
)

tuner.search(x_train, y_train, epochs=20, validation_data=(x_val, y_val))

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

print("Best hyperparameters:", best_hyperparameters.values)

## (c) Melhor desempenho da CNN

In [ ]:
lb = LabelBinarizer()
y_train = lb.fit_transform(train.labels.ravel())
y_val = lb.fit_transform(val.labels.ravel())
y_test = lb.transform(test.labels.ravel())

input_tensor = Input(shape=(28, 28, 3))

network = Conv2D(filters=256, kernel_size=2, strides=1, padding='same')(input_tensor)
network = MaxPooling2D((1,1))(network)
network = BatchNormalization()(network)
network = ReLU()(network)

network = GlobalAveragePooling2D()(network)
output_tensor = Dense(8, activation='softmax')(network)

model = Model(inputs=input_tensor, outputs=output_tensor)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint('best_model_cnn1.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001)

epochs = 30
batch_size = 32

history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpoint, early_stopping])

Plote matriz de confusão e 5 instâncias erradas


In [ ]:
#test_loss, test_acc = model.evaluate(x_test, y_test)
#print(f'Teste de Acurácia: {test_acc:.3f}, Teste de Perda: {test_loss:.3f}')

y_pred = model.predict(x_test)
y_pred_indices = np.argmax(y_pred, axis=1)

y_test_ori = test.labels.ravel()

#cm = confusion_matrix(y_test_ori, y_pred_indices)
#plt.figure(figsize=(8, 6))
#sns.heatmap(cm, annot=True, cmap='Reds', fmt='g', xticklabels=['0','1', '2', '3', '4', '5', '6','7'], yticklabels=['0','1', '2', '3', '4', '5', '6','7'])
#plt.xlabel('Predicted')
#plt.ylabel('True')
#plt.show()


errors = np.where(y_pred_indices != y_test_ori)[0]

selected_errors = np.random.choice(errors, 5, replace=False)

fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 5))


for i, error_index in enumerate(selected_errors):
    ax_image = axes[i, 0]
    ax_image.imshow(x_test[error_index].reshape(28, 28, 3))
    ax_image.set_title('')
    ax_image.axis('off')


    ax_label = axes[i, 1]
    true_label = y_test_ori[error_index]
    pred_label = y_pred_indices[error_index]
    label_text = f"True: {lb.classes_[true_label]}\nPredict: {lb.classes_[pred_label]}"
    ax_label.text(0.01, 0.5, label_text, ha='center', va='center', fontsize=12, family='monospace')
    ax_label.axis('off')


    ax_probs1 = axes[i, 2]
    predicted_probs = y_pred[error_index]
    probs_text1 = "\n".join([f"{lb.classes_[j]}: {prob * 100:.2f}%" for j, prob in enumerate(predicted_probs[:4])])
    ax_probs1.text(0.01, 0.5, probs_text1, ha='center', va='center', fontsize=12, family='monospace')
    ax_probs1.axis('off')


    ax_probs2 = axes[i, 3]
    probs_text2 = "\n".join([f"{lb.classes_[j] + 4}: {prob * 100:.2f}%" for j, prob in enumerate(predicted_probs[4:])])
    ax_probs2.text(0.01, 0.5, probs_text2, ha='center', va='center', fontsize=12, family='monospace')
    ax_probs2.axis('off')

plt.tight_layout()
plt.show()

##(d) CNN profunda

In [ ]:
from medmnist import BloodMNIST

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelBinarizer

#Criação do bloco residual
def residual_block(x, filters, kernel_size=3, stride=1):

    #faz uma cópia da entrada para ser somada com a saída depois
    shortcut = x

    #Arq da resnet, camada conv -> BN -> relu -> conv -> BN
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    #torna as dimensões iguais, caso o salto seja diferente de 1 ou tenha num de filtros dierentes
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)

    #soma saída e entrada e passa na relu
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

train = BloodMNIST(split="train", download= True, size = 28)

val = BloodMNIST(split="val", download= True, size = 28)

test = BloodMNIST(split="test", download= True, size = 28)

#entrada da rede
input_tensor = Input(shape=(28, 28, 3))

#primeira camada conv
x = Conv2D(64, kernel_size=7, strides=2, padding='same')(input_tensor)
x = BatchNormalization()(x)
x = ReLU()(x)

#Blocos residuais
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 256, stride=1)
x = residual_block(x, 256, stride=1)
x = residual_block(x, 256, stride=1)
x = residual_block(x, 256, stride=1)
x = residual_block(x, 256, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)
x = residual_block(x, 128, stride=1)

#camada de pooling e saída
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(8, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#model.summary()

#normalização da entrada (entre 0 e 1)
x_train = train.imgs / 255
x_val = val.imgs / 255
x_test = test.imgs / 255

#torna as labels no formato de vetor binário
lb = LabelBinarizer()
y_train = lb.fit_transform(train.labels.ravel())
y_val = lb.fit_transform(val.labels.ravel())
y_test = lb.transform(test.labels.ravel())

#callback pra early_stopping e para salvar melhor modelo com base em val
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=8, min_delta=0.0001)

epochs = 30
batch_size = 32

#treinamento
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_val, y_val),
                    callbacks=[checkpoint, early_stopping])

Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Epoch 1/30
  3/374 [..............................] - ETA: 2:01:02 - loss: 4.2347 - accuracy: 0.1875

KeyboardInterrupt: 

In [ ]:
model = load_model('best_model.keras')

#avaliar o modelo no conjunto de testes
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Teste de Acurácia: {test_acc:.3f}, Teste de Perda: {test_loss:.3f}')